## Atlas Cohort Exchange - Cohort Definition Export
This notebook is meant for exporting the cohort definition from the central Atlas site for subsequent import into external Atlas sites.

This is step 1 in the overall Atlas Cohort Exchange process:
1. Export cohort definition as a JSON file - this step (_see Atlas Cohort Exchange - Admin.docx_)
2. Create the same cohort definition at remote Atlas site using the **AtlasCohortExchange R** package, then create inclusion report (_see Atlas Cohort Exchange - User Guide.docx_)
3. Export cohort inclusion report results to JSON file using the **AtlasCohortExchange R** package (_see Atlas Cohort Exchange - User Guide.docx_)
4. Import cohort inclusion report results into central Atlas instance using the **AtlasCohortExchange-ResultImport** Jupyter notebook (_see Atlas Cohort Exchange - Admin.docx_)

## Parameters - check/modify before running rest of notebook
These are the parameters to be used when connecting to the central Atlas database to export the cohort definition.

###### Database parameters
The parameters needed for connecting to the relevant database and (webapi) schema of the central Atlas site.

###### Cohort definition ID
The cohort_definition_id of the central Atlas instance for the cohort definition to export and share with external Atlas site(s).

###### Path to the JSON file & file name for JSON file
The directory path to the JSON file in which the cohort definition should be saved, and the file name to use.

In [3]:
# ##### Database parameters ##### 
dbDbms = 'postgresql' #  ONLY POSTGRESQL SUPPORTED CURRENTLY
dbAddress = '34.251.82.109'
dbDatabase = 'omop1'
dbSchema = 'webapi'
dbUser = 'atlasuser'
dbPassword='ec2-psql'

# ##### Cohort definition ID ##### 
cohortId=146

# ##### Path to and name of the JSON file: ##### 
# #####   jsonFileDirecory: the path where the JSON file should be saved
# #####   jsonFileName: defaults to 'cohort_xxx_def.json', where xxx is the cohort definition id ##### 
jsonFileDirecory = 'C:/dwn/'
jsonFileName = 'cohort_' + str(cohortId) + '_def.json'

## Code - export the cohort definition
Once the parameters above have been checked/modified, please run the remainder of the notebook cells to export the cohort definition to a JSON file.

In [4]:
# Import necessary libraries
import pandas as pd
import psycopg2
import pymssql
import json
from sqlalchemy import create_engine

In [5]:
# Create connection string - ONLY POSTGRESQL SUPPORTED CURRENTLY
if dbDbms == 'postgresql':
    dbConnectionString = 'postgresql+psycopg2://' + dbUser + ':' + dbPassword + '@' + dbAddress + '/' + dbDatabase
else:
    dbConnectionString = ''

In [6]:
# Identify database
engine = create_engine(dbConnectionString)

In [7]:
# load cohort_definition
sql = 'select * from ' + dbSchema + '.cohort_definition where id=' + str(cohortId)
cd=pd.read_sql(sql, engine)

In [8]:
# load cohort_definition_details
sql = 'select * from ' + dbSchema + '.cohort_definition_details where id=' + str(cohortId)
cdd=pd.read_sql(sql, engine)

In [9]:
# Combine the two JSON outputs
out_str = '[{"cohort_definition": ' + cd.to_json(date_format='iso') + '},'
out_str += '{"cohort_definition_details": ' + cdd.to_json(date_format='iso') + '}]'

In [10]:
# Write the JSON file
jsonFilePath = jsonFileDirecory + jsonFileName
with open(jsonFilePath, 'w') as data_file:    
    data_file.write(out_str)